In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from tqdm import tqdm
import csv
import pickle
import numpy as np
from pronto import Ontology
import spacy
import faiss
from tqdm import tqdm
import warnings
import gc
import re
import pandas
import faiss
import pickle
import spacy
import numpy as np
from fuzzywuzzy import fuzz

/nfs/production/literature/santosh_tirunagari/transformers_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-13 13:11:41.882338: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-13 13:11:46.133804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 13:11:47.713632: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been

In [2]:
# Filenames

# santosh path__ = "/nfs/production/literature/santosh_tirunagari/BACKUP/work/github/source_data/knowledge_base/bao/BAO.csv"
path__ = "/homes/ines"
INPUT_FILENAME = path__ + "/dictionary/BAO.csv"
OUTPUT_PICKLE_FILENAME = path__ + "/dictionary/bao.pkl"
OUTPUT_LIST = path__ + "/dictionary/bao_list.txt"
FAISS_INDEX_FILENAME = path__ + "/dictionary/bao_terms.index"
# OUTPUT_INDEXED_TERMS_FILENAME = path__+"work/github/ML_annotations/normalisation/dictionary/bao_indexed_terms.pkl"


In [3]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Sentences we want sentence embeddings for
sentences = ['diabetes', 'type 2']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('tavakolih/all-MiniLM-L6-v2-pubmed-full')# pritamdeka/S-PubMedBert-MS-MARCO
model = AutoModel.from_pretrained('tavakolih/all-MiniLM-L6-v2-pubmed-full')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

tokenizer_config.json: 100%|██████████████████| 564/564 [00:00<00:00, 917kB/s]
vocab.txt: 100%|███████████████████████████| 232k/232k [00:00<00:00, 1.58MB/s]
tokenizer.json: 100%|██████████████████████| 712k/712k [00:00<00:00, 2.41MB/s]
config.json: 100%|███████████████████████████| 720/720 [00:00<00:00, 1.50MB/s]
pytorch_model.bin: 100%|█████████████████| 90.9M/90.9M [00:02<00:00, 35.0MB/s]


Sentence embeddings:
tensor([[-2.7119e-02,  9.8441e-02, -4.8616e-02,  1.2813e-01, -3.2804e-02,
          2.5970e-02,  1.5071e-01,  5.6532e-02, -1.4452e-02, -1.0685e-02,
         -5.9629e-02,  6.7335e-02, -7.7009e-02, -2.6510e-02, -8.2213e-02,
         -7.1197e-02, -9.9482e-03, -3.2808e-02, -4.1267e-03,  1.3900e-02,
          7.8650e-02,  6.7884e-02,  7.2451e-02,  3.8066e-02,  2.2594e-03,
          5.1958e-03,  6.6761e-02, -5.6698e-02, -5.5391e-02, -5.5787e-02,
         -4.6120e-02,  3.8803e-02,  9.4903e-03,  6.7905e-03, -7.3449e-02,
          8.6693e-02,  8.5998e-03, -1.2169e-02, -8.1756e-02, -6.2011e-02,
          4.1464e-02, -5.4746e-02, -1.2144e-02,  7.2727e-02,  4.6662e-02,
         -6.0085e-03, -7.6413e-02,  5.9721e-02,  5.6087e-02,  8.2868e-02,
         -5.2568e-02, -2.4164e-02,  3.5872e-02, -5.4179e-03,  9.6168e-02,
         -5.0484e-02, -9.6207e-02, -5.9018e-02, -5.0531e-02, -1.0543e-01,
         -2.1302e-02,  2.0079e-02, -3.4305e-03,  2.3637e-02,  6.8764e-02,
         -5.5543e

In [4]:
warnings.simplefilter("ignore")



def create_quantized_index(embeddings_np, d):
    """Create a trained IVFPQ index."""
    nlist = 1000
    m = 32
    quantizer = faiss.IndexFlatL2(d)
    index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
    index.train(embeddings_np)
    return index


#
def get_average_embeddings_batched(terms):
    """Return average embeddings for terms."""
    docs = list(nlp.pipe(terms))
    embeddings = []

    for doc in docs:
        # Filtering out tokens without vectors or with unexpected vector sizes
        valid_vectors = [token.vector for token in doc if token.has_vector and token.vector_norm != 0 and token.vector.shape[0] == 300]

        # If no valid vectors, append a zero vector
        if len(valid_vectors) == 0:
            embeddings.append(np.zeros((300,)))
        else:
            average_embedding = np.mean(valid_vectors, axis=0)
            embeddings.append(average_embedding)

    return embeddings


def get_average_embeddings_batched_transformers(sentences, model_name="tavakolih/all-MiniLM-L6-v2-pubmed-full"):
    """Return average embeddings for sentences using a Transformers model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling (mean pooling function is used here)
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]  # First element of model_output contains token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings


In [5]:
def process_column_content(s):
    """Clean and strip unwanted characters and split by pipe if present."""
    # First, clean the string by removing specific patterns
    cleaned = s.strip().lower()
    
    # Check if the cleaned string contains a pipe symbol and split if it does
    if '|' in cleaned:
        return cleaned.split('|')
    else:
        return cleaned


df = pd.read_csv(INPUT_FILENAME, usecols=['Class ID', 'Preferred Label', 'Synonyms', 'Definitions', 'alternative term'], 
                 sep=',', engine='python', on_bad_lines='skip')


term_to_id = {}
embeddings = []  
indexed_terms = []

In [6]:
flattened_data = []
for _, row in df.iterrows():
    term_id = row['Class ID']
    for col in ['Preferred Label', 'Synonyms', 'Definitions', 'alternative term']:
        term_names = row[col]
        if pd.notnull(term_names):  # Check if the term_name is not NaN
            processed_terms = process_column_content(term_names)
            if isinstance(processed_terms, list):
                for term in processed_terms:
                    flattened_data.append((term_id, term))
            else:
                flattened_data.append((term_id, processed_terms))

# Convert flattened data to a DataFrame for easier manipulation
flattened_df = pd.DataFrame(flattened_data, columns=['Class ID', 'Term Name'])

In [7]:
flattened_df

,Class ID,Term Name
0,http://purl.obolibrary.org/obo/CHEBI_50444,adenosine phosphodiesterase inhibitor
1,http://purl.obolibrary.org/obo/CHEBI_131787,dopamine receptor d2 antagonist
2,http://purl.obolibrary.org/obo/CHEBI_131787,d2r antagonist
3,http://purl.obolibrary.org/obo/CHEBI_131787,d2 receptor antagonist
4,http://purl.obolibrary.org/obo/CHEBI_131789,runx1 inhibitor
...,...,...
33353,http://purl.obolibrary.org/obo/DOID_3953,adrenal cancer
33354,http://purl.obolibrary.org/obo/DOID_3953,tumor of the adrenal gland
33355,http://purl.obolibrary.org/obo/DOID_3953,malignant neoplasm of adrenal gland
33356,http://purl.obolibrary.org/obo/DOID_3953,malignant adrenal tumor


In [8]:
embeddings = []
term_to_id = {}
indexed_terms = []

BATCH_SIZE = 100
term_batches = []
id_batches = []
current_batch_terms = []
current_batch_ids = []

for _, row in tqdm(flattened_df.iterrows(), total=flattened_df.shape[0], desc="Processing terms"):
    term_id = row['Class ID']
    term_name = row['Term Name']

    # Process the term_name
    term_name = process_column_content(term_name)

    # Check for empty or single character terms and skip them
    if not term_name or len(term_name) <= 1:
        continue

    current_batch_terms.append(term_name)
    current_batch_ids.append(term_id)

    if len(current_batch_terms) == BATCH_SIZE:
        term_batches.append(current_batch_terms)
        id_batches.append(current_batch_ids)
        current_batch_terms = []
        current_batch_ids = []

# Catch any remaining terms not added to a batch
if current_batch_terms:
    term_batches.append(current_batch_terms)
    id_batches.append(current_batch_ids)

for term_batch, id_batch in tqdm(zip(term_batches, id_batches), total=len(term_batches), desc="Generating Embeddings"):
    batch_embeddings = get_average_embeddings_batched_transformers(term_batch)
    
    for term, term_id, embedding in zip(term_batch, id_batch, batch_embeddings):
        norm = np.linalg.norm(embedding)

        # Check if the embedding is a zero vector
        if norm == 0:
            print(f"Term '{term}' with ID '{term_id}' has a zero vector.")

        # Normalizing the vector
        normalized_embedding = embedding if norm == 0 else embedding / norm
        embeddings.append(normalized_embedding)
        term_to_id[term] = term_id
        indexed_terms.append(term)

        # Clear out the current batch to free up memory
    del term_batch, id_batch, batch_embeddings
    gc.collect()

# Assuming we have already calculated sentence_embeddings somewhere in the script
d = embeddings[0].shape[0] if embeddings else 0  # Dynamically get the dimension
embeddings_np = np.array(embeddings).astype('float32')
index = create_quantized_index(embeddings_np, d)
index.add(embeddings_np)

# Free up memory after using embeddings_np
del embeddings, embeddings_np
gc.collect()

print("Saving quantized faiss index...")
faiss.write_index(index, FAISS_INDEX_FILENAME)

# print("Saving term to ID mapping...")
# with open(OUTPUT_PICKLE_FILENAME, "wb") as outfile:
#     pickle.dump(term_to_id, outfile)

print("Saving term to ID mapping and indexed terms...")
with open(OUTPUT_PICKLE_FILENAME, "wb") as outfile:
    pickle.dump({"term_to_id": term_to_id, "indexed_terms": indexed_terms}, outfile)


print("Writing terms to a txt file...")
with open(OUTPUT_LIST, "w") as txt_file:
    for term in term_to_id.keys():
        txt_file.write(term + "\n")
print("Finished")

Generating Embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 334/334 [10:00<00:00,  1.80s/it]
WARNING clustering 33342 points to 1000 centroids: please provide at least 39000 training points


Saving quantized faiss index...
Saving term to ID mapping and indexed terms...
Writing terms to a txt file...
Finished


In [9]:
# Load the term to ID mapping and indexed terms
with open(OUTPUT_PICKLE_FILENAME, "rb") as infile:
    data = pickle.load(infile)
    term_to_id = data["term_to_id"]
    indexed_terms = data["indexed_terms"]

# Load the FAISS index
index = faiss.read_index(FAISS_INDEX_FILENAME)


def retrieve_similar_terms(query, k=5):
    """Retrieve top k similar terms given a query."""
    query = query.lower()  # Convert query to lowercase
    query_embedding = get_average_embeddings_batched_transformers([query])[0].numpy()  # Get average embedding of the query

    # Normalize the query embedding
    norm = np.linalg.norm(query_embedding)
    query_embedding = query_embedding if norm == 0 else query_embedding / norm
    query_embedding = query_embedding.reshape(1, -1).astype('float32')

    # Search the index
    D, I = index.search(query_embedding, k)

    similar_terms = []
    for i in range(k):
        term = indexed_terms[I[0][i]]
        score = D[0][i]
        term_id = term_to_id[term]
        similar_terms.append((term, term_id, score))

    return similar_terms

def retrieve_similar_terms_with_fuzzy(query, k):
    """Retrieve k terms similar to the query."""
    query_embedding = get_average_embeddings_batched_transformers([query])[0].numpy()  # Get average embedding of the query

    # Normalize the query embedding
    norm = np.linalg.norm(query_embedding)
    query_embedding = query_embedding if norm == 0 else query_embedding / norm
    query_embedding = query_embedding.reshape(1, -1).astype('float32')

    # Search the index
    D, I = index.search(query_embedding, k)

    # Retrieve the terms from the indexed_terms list
    candidate_terms = [indexed_terms[i] for i in I[0]]

    # Get fuzzy matching scores for these terms
    scores = [fuzz.ratio(query, term) for term in candidate_terms]

    # Pair up terms with their scores
    term_score_pairs = list(zip(candidate_terms, scores))

    # Rank these pairs based on scores
    ranked_term_score_pairs = sorted(term_score_pairs, key=lambda x: x[1], reverse=True)

    return ranked_term_score_pairs[:k]


In [10]:
def show_suggestions(term):
    query = term
    results = retrieve_similar_terms(query, 5)

    for term, term_id, score in results:
        print(f"Term: {term}, ID: {term_id}, Score: {score}")

In [11]:
# Example usage

show_suggestions('Surface plasmon resonating')

Term: surface plasmon resonance, ID: http://www.bioassayontology.org/bao#BAO_0000054, Score: 0.5017558336257935
Term: one of several models of surface plasmon resonance (spr) instruments., ID: http://www.bioassayontology.org/bao#BAO_0150085, Score: 0.64713454246521
Term: for surface plasmon resonance measurements, target protein is immobilized onto a solid surface via covalent or noncovalent interaction. binding of small molecule compounds changes the refractive index of the immobilized target, which can be detected by optical sensing. shift in wavelength of light is proportional to mass changes near the sensor surface, with sensitivity to detect mass change being ~300 da., ID: http://www.bioassayontology.org/bao#BAO_0000054, Score: 0.7980142831802368
Term: resonant waveguide grating, ID: http://www.bioassayontology.org/bao#BAO_0000069, Score: 0.9245482683181763
Term: excitation wavelength, ID: http://www.bioassayontology.org/bao#BAO_0000566, Score: 0.9980339407920837


In [12]:
show_suggestions('SPR analysis')

Term: spr, ID: http://www.bioassayontology.org/bao#BAO_0000054, Score: 0.509190559387207
Term: quantitative pcr, ID: http://www.bioassayontology.org/bao#BAO_0003031, Score: 0.8667311668395996
Term: real-time pcr, ID: http://www.bioassayontology.org/bao#BAO_0002084, Score: 0.8763004541397095
Term: taqman real-time pcr assays (thermo-fisher), ID: http://www.bioassayontology.org/bao#BAO_0140015, Score: 0.8813551068305969
Term: physical tracer, ID: http://purl.obolibrary.org/obo/CHEBI_35208, Score: 0.893916130065918


In [13]:
show_suggestions('TR-FRET')

Term: tr-fret, ID: http://www.bioassayontology.org/bao#BAO_0000004, Score: 0.3012160360813141
Term: cret, ID: http://www.bioassayontology.org/bao#BAO_0000462, Score: 0.7369927763938904
Term: trupath, ID: http://www.bioassayontology.org/bao#BAO_0010081, Score: 0.7620601654052734
Term: trna, ID: http://www.bioassayontology.org/bao#BAO_0000276, Score: 0.7682546973228455
Term: thale-cress, ID: http://purl.obolibrary.org/obo/NCBITaxon_3702, Score: 0.7849703431129456


In [14]:
show_suggestions('Radioligand displacement assay')

Term: ion redistribution assay, ID: http://www.bioassayontology.org/bao#BAO_0000819, Score: 0.7139953374862671
Term: metal ion redistribution assay, ID: http://www.bioassayontology.org/bao#BAO_0000162, Score: 0.7736241817474365
Term: primary assay sar, ID: http://www.bioassayontology.org/bao#BAO_0000031, Score: 0.7854743003845215
Term: signal transduction assay, ID: http://www.bioassayontology.org/bao#BAO_0003002, Score: 0.8055042624473572
Term: this can be moved to directly under 'assay measurement'., ID: http://www.bioassayontology.org/bao#BAO_0002069, Score: 0.8106790781021118
